    Copyright (c) 2018,  Bosch Software Innovations GmbH.

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.

# Setup

Import of all the basic modules ([pandas](https://pandas.pydata.org/)). See [plotting](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) for help on charts.

In [ ]:
%matplotlib inline
import os
import pandas as pd

Common Settings

In [ ]:
BASE_PATH = os.path.join('build', 'bin')

Common Code

In [ ]:
def read_csv(filepath, columns = None):
    dataframe = pd.read_csv(os.path.join(BASE_PATH, filepath))
    if columns is not None:
        assert set(dataframe.columns) == set(columns)
    return dataframe

In [ ]:
def calc_duration(dataframe, start_column, end_column, target_column):
    dataframe[target_column] = dataframe.apply(lambda row: row[end_column] - row[start_column], axis=1)
    dataframe.drop([start_column, end_column], axis=1, inplace=True)

In [ ]:
def delete_column(dataframe, column):
    dataframe.drop([column], axis=1, inplace=True)

In [ ]:
def duration(dataframe, start_column, end_column, target_column):
    dataframe[target_column] = dataframe.apply(lambda row: row[end_column] - row[start_column], axis=1)

In [ ]:
def scale_value(dataframe, source_column, target_column, factor, ndigits=None):
    dataframe[target_column] = dataframe.apply(lambda row: factor * row[source_column], axis=1)
    if ndigits is not None:
        dataframe[target_column] = dataframe.apply(lambda row: round(row[target_column], ndigits), axis=1)
    if source_column != target_column:
        dataframe.drop([source_column], axis=1, inplace=True)

In [ ]:
def progress_column(subject):
    return lambda i: subject + '_' + str(i) + ' (ms)'

In [ ]:
percent = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
def calc_write_progress(dataframe, progress_column, target_column):
    for i in percent:
        duration(dataframe, progress_column(i-10), progress_column(i), target_column(i))
    for i in [0] + percent:
        delete_column(dataframe, progress_column(i))

# Benchmarks

## Write

### 1) different SQLite modi: performance vs. "stability"

In [ ]:
#data1_columns = None
#data1 = read_csv('sqlite-db-settings-benchmark.csv', data1_columns)

### 2) small messages

In [ ]:
data2 = read_csv('small_messages_benchmark.csv', [
    'description',
    'number of messages',
    'message blob size (bytes)',
    'transaction size',
    'start writing time (ms)',
    'write_throughput_0 (ms)',
    'write_throughput_10 (ms)',
    'write_throughput_20 (ms)',
    'write_throughput_30 (ms)',
    'write_throughput_40 (ms)',
    'write_throughput_50 (ms)',
    'write_throughput_60 (ms)',
    'write_throughput_70 (ms)',
    'write_throughput_80 (ms)',
    'write_throughput_90 (ms)',
    'write_throughput_100 (ms)',
    'end writing time (ms)',
    'start indexing time (ms)',
    'end indexing time (ms)',
    'disk usage (bytes)'
])

calc_duration(data2, 'start writing time (ms)', 'end writing time (ms)', 'writing time (ms)')
calc_duration(data2, 'start indexing time (ms)', 'end indexing time (ms)', 'indexing time (ms)')
scale_value(data2, 'disk usage (bytes)', 'disk usage (MB)', factor=1/1024/1024)

data2['disk io (messages / s)'] = data2.apply(lambda row: row['number of messages'] / row['writing time (ms)'] * 1000, axis=1)
data2['disk io (MB / s)'] = data2.apply(lambda row: row['number of messages'] * row['message blob size (bytes)'] / 1024 / 1024 / row['writing time (ms)'] * 1000, axis=1)

throughput_abs = progress_column('write_throughput')
throughput_duration = progress_column('write_duration')
calc_write_progress(data2, throughput_abs, throughput_duration)


In [ ]:
data2_aggregated = data2.groupby('description').median().sort_values(by='message blob size (bytes)')
data2_aggregated

#### Throughput
This diagram shows how long it took to write each 10% slice of the messages.

In [ ]:
data2_aggregated[[throughput_duration(i) for i in percent]].T.plot(kind='bar')

In [ ]:
data2_aggregated.plot(kind='bar', x='message blob size (bytes)', y='writing time (ms)');

In [ ]:
data2_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk usage (MB)');

In [ ]:
data2_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk io (messages / s)');

In [ ]:
data2_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk io (MB / s)');

### 3) big messages

In [ ]:
data3 = read_csv('big_messages_benchmark.csv', [
    'description',
    'number of messages',
    'message blob size (bytes)',
    'transaction size',
    'start writing time (ms)',
    'write_throughput_0 (ms)',
    'write_throughput_10 (ms)',
    'write_throughput_20 (ms)',
    'write_throughput_30 (ms)',
    'write_throughput_40 (ms)',
    'write_throughput_50 (ms)',
    'write_throughput_60 (ms)',
    'write_throughput_70 (ms)',
    'write_throughput_80 (ms)',
    'write_throughput_90 (ms)',
    'write_throughput_100 (ms)',
    'end writing time (ms)',
    'start indexing time (ms)',
    'end indexing time (ms)',
    'disk usage (bytes)'
])

calc_duration(data3, 'start writing time (ms)', 'end writing time (ms)', 'writing time (ms)')
calc_duration(data3, 'start indexing time (ms)', 'end indexing time (ms)', 'indexing time (ms)')

scale_value(data3, 'disk usage (bytes)', 'disk usage (MB)', factor=1/1024/1024)

data3['disk io (messages / s)'] = data3.apply(lambda row: row['number of messages'] / row['writing time (ms)'] * 1000, axis=1)
data3['disk io (MB / s)'] = data3.apply(lambda row: row['number of messages'] * row['message blob size (bytes)'] / 1024 / 1024 / row['writing time (ms)'] * 1000, axis=1)

throughput_abs = progress_column('write_throughput')
throughput_duration = progress_column('write_duration')
calc_write_progress(data3, throughput_abs, throughput_duration)

In [ ]:
data3_aggregated = data3.groupby('description').median().sort_values(by='message blob size (bytes)')
data3_aggregated

#### Throughput
This diagram shows how long it took to write each 10% slice of the messages.

In [ ]:
data3_aggregated[[throughput_duration(i) for i in percent]].T.plot(kind='bar')

In [ ]:
data3_aggregated.plot(kind='bar', x='message blob size (bytes)', y='writing time (ms)');

In [ ]:
data3_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk usage (MB)');

In [ ]:
data3_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk io (messages / s)');

In [ ]:
data3_aggregated.plot(kind='bar', x='message blob size (bytes)', y='disk io (MB / s)', sort_columns=True);

### 4) combination of small and big messages

In [ ]:
data4 = read_csv('mixed_messages_benchmark.csv', [
    'description',
    'number of small messages',
    'small message blob size (bytes)',
    'number of medium messages',
    'medium message blob size (bytes)',
    'number of big messages',
    'big message blob size (bytes)',
    'transaction size',
    'start writing time (ms)',
    'write_throughput_0 (ms)',
    'write_throughput_10 (ms)',
    'write_throughput_20 (ms)',
    'write_throughput_30 (ms)',
    'write_throughput_40 (ms)',
    'write_throughput_50 (ms)',
    'write_throughput_60 (ms)',
    'write_throughput_70 (ms)',
    'write_throughput_80 (ms)',
    'write_throughput_90 (ms)',
    'write_throughput_100 (ms)',
    'end writing time (ms)',
    'start indexing time (ms)',
    'end indexing time (ms)',
    'disk usage (bytes)'
])

calc_duration(data4, 'start writing time (ms)', 'end writing time (ms)', 'writing time (ms)')
calc_duration(data4, 'start indexing time (ms)', 'end indexing time (ms)', 'indexing time (ms)')

scale_value(data4, 'disk usage (bytes)', 'disk usage (MB)', factor=1/1024/1024)

data4['disk io (messages / s)'] = data4.apply(lambda row: (row['number of small messages'] + row['number of medium messages'] + row['number of big messages']) / row['writing time (ms)'] * 1000, axis=1)
data4['disk io (MB / s)'] = data4.apply(lambda row: (row['number of small messages'] * row['small message blob size (bytes)'] + row['number of medium messages'] * row['medium message blob size (bytes)'] + row['number of big messages'] * row['big message blob size (bytes)']) / 1024 / 1024 / row['writing time (ms)'] * 1000, axis=1)


throughput_abs = progress_column('write_throughput')
throughput_duration = progress_column('write_duration')
calc_write_progress(data4, throughput_abs, throughput_duration)

data4

#### Throughput
This diagram shows how long it took to write each 10% slice of the messages.

In [ ]:
data4.median()[[throughput_duration(i) for i in percent]].T.plot(kind='bar')

### 5) create index while or after writing file

In [ ]:
#data5_columns = None
#data5 = read_csv('sqlite-index-benchmark.csv', data5_columns)

### 6) huge file

In [ ]:
#data6_columns = None
#data6 = read_csv('sqlite-huge-file-benchmark.csv', data6_columns)

## Read

...